<a href="https://colab.research.google.com/github/DinhoVCO/Mineria-de-datos/blob/main/Final_ALGORITMO_APRIORI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ALGORITMO APRIORI**
---
**UNIVERSIDAD NACIONAL DE SAN ANTONIO ABAD DEL CUSCO**


> ESCUELA PROFESIONAL: Ing. Informatica y de Sistemas



> CURSO: Minería de datos💻


> ALUMNO : Vega Centeno Olivera, Ronaldinho🧑









##**1.-Instalar bibliotecas**

In [ ]:
import numpy as np #para procesar el dataset
import statistics as stats #para hallar medidas estadisticas

## **2.-Importar datos**

In [ ]:
#Importamos el archivo .npy en la variable data de tipo numpy.ndarray 0-dimensional
data = np.load('spotify.npy',allow_pickle=True,fix_imports=True)
#convertimos a una lista -> obtenemos un diccionario
data=data.tolist()
#Cantidad de playlist
print("Total de PlayLists:",len(data))

Total de PlayLists: 10000


In [ ]:
#Canciones de la playlist n° 0 sin repeticiones y ordenadas alfabeticamente
set(data[0][0:5])

{'Crazy In Love',
 "It Wasn't Me",
 'Lose Control (feat. Ciara & Fat Man Scoop)',
 'Rock Your Body',
 'Toxic'}

##**3.- Preprocesar (lista de listas)**

In [ ]:
#Generamos una lista de canciones unicas y de playlist sin canciones repetidas
def dataset_preprocessing(data):
  playlist=[]
  songs=set()
  for i in data:
    playlist.append(set(data[i]))
    songs|=set(data[i])
  
  songs=[[i] for i in list(songs)]
  songs.sort()
  return list(map(frozenset,songs)), playlist

In [ ]:
songs, playlist = dataset_preprocessing(data)
print('Total de canciones distintas:', len(songs))
print('Total de Playlists(transaccciones):', len(playlist))

Total de canciones distintas: 132920
Total de Playlists(transaccciones): 10000


In [ ]:
#Tres canciones
songs[:3]

[frozenset({'!Dios Mio! El Diablo Es Muy Picante'}),
 frozenset({'"*!*!" What U Heard'}),
 frozenset({'"2014: A Pesach Medley"'})]

In [ ]:
#Plailist n°999
playlist[999]

{'A Quién Le Importa',
 'Amor De Pobre - No Tengo Dinero (Hip-Hop Version)',
 'El Hombre Es Como El Oso',
 'El Mario de Tu Mujer (feat. Don Miguelo)',
 'El Regeton',
 'El Tao Tao',
 'Juana La Cubana',
 'La Bomba',
 'La Mesa Que Mas Aplauda (Za Za Za)',
 'La Vaca',
 'Muévelo',
 'Party In Miami',
 'Que Lo Que',
 'Tu Pun Pun',
 'Un Ladrón'}

## **4.-Definir umbrales** 🟠⏮ 

In [710]:
frequencies=dict()
for i in playlist:
  for j in i:
    frequencies[j] =frequencies.get(j, 0) + 1
frecuencias=list(frequencies.values())
frecuencias.sort()
#print(frecuencias)
print(len(frecuencias))

132920


In [ ]:
print("Media aritmética: ",stats.mean(frecuencias))
print("Media armónica: ",stats.harmonic_mean(frecuencias)) 
print("Mediana: ",stats.median(frecuencias))
print("Mediana baja: ",stats.median_low(frecuencias))
print("Mediana alta: ",stats.median_high(frecuencias))
print("Percentil 50:",stats.median_grouped(frecuencias)) 
print("Moda: ",stats.mode(frecuencias)) 
print("Varianza poblacional : ",stats.pvariance(frecuencias)) 
print("Desviación estándar: ",stats.pstdev(frecuencias))
print("Varianza muestral :",stats.variance(frecuencias)) 

Media aritmética:  4.910028588624737
Media armónica:  1.4028898793768465
Mediana:  1.0
Mediana baja:  1
Mediana alta:  1
Percentil 50: 1.3384321344317307
Moda:  1
Varianza poblacional :  254.75020336963107
Desviación estándar:  15.960896070384992
Varianza muestral : 254.75211995193587


In [ ]:
lon_min = 2 # Número mínimo de productos a considerar en las reglas
veces_minimas_aparece_en_lista=100
soporte_min = veces_minimas_aparece_en_lista / len(playlist)
confianza_min =0.5
sustento_min =1.5

## **5.-Ejecutar algoritmo** 

### **5.1. Get frequent items**

#### get_first_frequent_items

In [ ]:
#Modulo optimo para obtener 1-itemsetfrequents
def get_first_frequent_items(transactions, k_itemSet,minSupport):
  frequencies=dict()
  for i in transactions:
    for j in i:
      frequencies[j] =frequencies.get(j, 0) + 1
  TotalTransactions = float(len(transactions))
  ItemSetFrequent = []
  supportItemSet = {}
  for key in frequencies:
      support = frequencies[key] / TotalTransactions  # Calcular soporte
      if support >= minSupport:  # Si el soporte es mayor que el soporte mínimo establecido
          ItemSetFrequent.insert(0,key)
          #ItemSetFrequent.append(key)
      supportItemSet[key] = support
  ItemSetFrequent.sort()
  ItemSetFrequent=[frozenset({i}) for i in ItemSetFrequent ]
  return ItemSetFrequent, supportItemSet

In [714]:
ItemSetFrequent_1,support = get_first_frequent_items(playlist[:2],songs,0.015)
print( 'Cantidad de  1-ItemSetFrequent: ', len(ItemSetFrequent_1))
print('Cantidad de support hallados:',len(support))
print('Canciones: ', ItemSetFrequent_1[:5])
print("Support de 'A Million Miles Away'=", support['A Million Miles Away'])


Cantidad de  1-ItemSetFrequent:  224
Cantidad de support hallados: 224
Canciones:  [frozenset({'A Million Miles Away'}), frozenset({'A Star Is Born'}), frozenset({'A Thousand Miles'}), frozenset({'A True Hero / A Star Is Born'}), frozenset({'A Whole New World'})]
Support de 'A Million Miles Away'= 0.5


#### get_frequent_items

In [ ]:
def get_frequent_items(transactions, k_itemSet,minSupport):
  frequencies=dict()
  for i in transactions:
    for j in k_itemSet:
      if (j<=i):
        frequencies[j] =frequencies.get(j, 0) + 1
  TotalTransactions = float(len(transactions))
  ItemSetFrequent = []
  supportItemSet = {}
  for key in frequencies:
      support = frequencies[key] / TotalTransactions  # Calcular soporte
      if support >= minSupport:  # Si el soporte es mayor que el soporte mínimo establecido
          ItemSetFrequent.insert(0,key)
          #ItemSetFrequent.append(key)
      supportItemSet[key] = support
  ItemSetFrequent.sort()
  return ItemSetFrequent, supportItemSet
  


In [715]:
frec,k = get_first_frequent_items(D[:2], C1, 0.015)
print(len(frec))
print(len(k))


224
224


In [716]:
frec,k = get_frequent_items(D[:2], C1, 0.001)
print(len(frec))
print(len(k))

224
224


### **5.2 Get Combination Apriori**

In [717]:
#GEnerar posibles conjuntos (k-ItemSet) a partir de mezclar los k-1 ItemSetFrequent
#Los ItemSet deben estar ordenados lexicográficamente
# Solo mezclamos los pares de itemsets que difieren en su último ítem.
def get_combination_apriori(ItemSetFrequent, k):
  ItemSet_k=[]
  tam=len(ItemSetFrequent)
  for i in range(tam):
    for j in range(i+1,tam):
      Arr1=list(ItemSetFrequent[i])[:k-1]
      Arr2=list(ItemSetFrequent[j])[:k-1]
      if(Arr1==Arr2):
        ItemSet_k.append(ItemSetFrequent[i]|ItemSetFrequent[j])
  return ItemSet_k


In [ ]:
C1, D = dataset_preprocessing(data)
print("Canciones:",len(C1))
print("playlists: ",len(D))
L1, supportData = get_first_frequent_items(D, C1, 0.015)
print(len(L1))
len(L1[:2])
len(C1)

Canciones: 132920
playlists:  10000
349


132920

In [ ]:
L1, supportData = get_first_frequent_items(D, C1, 0.015)
print(len(L1))
len(L1[:2])

349


2

In [ ]:
L=[L1]
Ck=get_combination_apriori(L[0],1)
len(Ck)

60726

In [ ]:
print(Ck[:3])

[frozenset({'0 To 100 / The Catch Up', "'Till I Collapse"}), frozenset({'1-800-273-8255', "'Till I Collapse"}), frozenset({"'Till I Collapse", '24K Magic'})]


### 5.3 Generacion de los k-itemSetsFrecuentes

In [ ]:
def get_frequent_itemsets(data, minSupport=0.2):  # Obtenga [L1, L2, L3 ..] lista de conjuntos de elementos frecuentes haciendo un bucle
    songs, playlists = dataset_preprocessing(data)
    L1, supportData = get_first_frequent_items(playlists, songs, minSupport)
    L = [L1]
    print("Total de canciones:",len(L[0]))
    k = 1
    while (len(L[k - 1]) > 0):  # Crear Ck
        print("k==",k," cantidad:",len(L[k-1]))
        Ck = get_combination_apriori(L[k - 1], k)
        print("Ck Generado:",len(Ck))
        Lk, supK = get_frequent_items(D, Ck, minSupport)
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

In [ ]:
L, supportData= get_frequent_itemsets(data,0.01)

Total de canciones: 838
k== 1  cantidad: 838
Ck Generado: 350703
k== 2  cantidad: 167
Ck Generado: 732
k== 3  cantidad: 19
Ck Generado: 17


In [623]:
len(L)

4

### 5.4 generar reglas

In [ ]:
import itertools 

In [658]:
def generar_reglas(conjunto):
  s = conjunto
  combinaciones= list(map(frozenset,chain.from_iterable(combinations(s, r) for r in range(1,len(s)))))
  reglas=[]
  for i in combinaciones:
    r=[i,conjunto-i]
    reglas.append(r)
  return reglas

In [659]:
generar_reglas(frozenset({1,2,3}))

[[frozenset({1}), frozenset({2, 3})],
 [frozenset({2}), frozenset({1, 3})],
 [frozenset({3}), frozenset({1, 2})],
 [frozenset({1, 2}), frozenset({3})],
 [frozenset({1, 3}), frozenset({2})],
 [frozenset({2, 3}), frozenset({1})]]

In [660]:
def print_reglas(reglas):
  for i in reglas:
    print(i[0],"=>",i[1])
    print("________________________________")

In [705]:
def print_reglas2(reglas,soportes,conf_min):
  resultado=[]
  for i in reglas:
    
    X_u_Y= soportes[i[0] | i[1]]
    if(len(i[0])==1):
      X =soportes[list(i[0])[0]]
    else:
      X =soportes[i[0]]
    confianza=X_u_Y/X
    if(confianza>=conf_min):
      r=[i[0],i[1]]
      resultado.append(r)
      print(i[0],"=>",i[1])
      print("confidence = ", confianza)
      print("________________________________")
  return(resultado)


In [691]:
print_reglas(generar_reglas(frozenset({1,2,3})))

frozenset({1}) => frozenset({2, 3})
________________________________
frozenset({2}) => frozenset({1, 3})
________________________________
frozenset({3}) => frozenset({1, 2})
________________________________
frozenset({1, 2}) => frozenset({3})
________________________________
frozenset({1, 3}) => frozenset({2})
________________________________
frozenset({2, 3}) => frozenset({1})
________________________________


In [706]:
def verificar_confianza(itemSetFrecuente,minconf,soportes):
  reglas=[]
  #print(len(itemSetFrecuente[0]))
  if(len(itemSetFrecuente[0])>1):
    for i in itemSetFrecuente:
      #print(i)
      *_,Y=i
      #print(Y)
      X=i-{Y}
      #print(X)
      soporte_X_u_Y=soportes[i]
      #print(soporte_X_u_Y)
      if(len(itemSetFrecuente[0])==2):
        soporte_X=soportes[list(X)[0]]
      else:
        soporte_X=soportes[X]
      #print(soporte_X)
      confianza=soporte_X_u_Y/soporte_X
      if(confianza>=minconf):
        #print(Y)
        #print(X)
        #print(confianza)
        reglas+=generar_reglas(i)
        #print_reglas2(generar_reglas(i),soportes,minconf)
    filtro_reglas=print_reglas2(reglas,soportes,minconf)
  else:
    print("No existen reglas")
  return filtro_reglas
    

In [707]:
R=verificar_confianza(L[2],0.5,supportData)

frozenset({'Congratulations', 'HUMBLE.'}) => frozenset({'iSpy (feat. Lil Yachty)'})
confidence =  0.5093457943925234
________________________________
frozenset({'Congratulations', 'iSpy (feat. Lil Yachty)'}) => frozenset({'HUMBLE.'})
confidence =  0.6728395061728395
________________________________
frozenset({'HUMBLE.', 'iSpy (feat. Lil Yachty)'}) => frozenset({'Congratulations'})
confidence =  0.7032258064516129
________________________________
frozenset({'HUMBLE.', 'DNA.'}) => frozenset({'Mask Off'})
confidence =  0.5263157894736842
________________________________
frozenset({'DNA.', 'Mask Off'}) => frozenset({'HUMBLE.'})
confidence =  0.9090909090909092
________________________________
frozenset({'Tunnel Vision', 'HUMBLE.'}) => frozenset({'XO TOUR Llif3'})
confidence =  0.7338129496402879
________________________________
frozenset({'Tunnel Vision', 'XO TOUR Llif3'}) => frozenset({'HUMBLE.'})
confidence =  0.7500000000000001
________________________________
frozenset({'HUMBLE.', 'XO 

In [708]:
def generar_todas_las_reglas(set_frecuentes,supportData,confi_min):
  k=len(L)-2
  verificar_confianza(set_frecuentes[k],confi_min,supportData)

In [709]:
generar_todas_las_reglas(L,supportData,0.5)

frozenset({'Congratulations', 'HUMBLE.'}) => frozenset({'iSpy (feat. Lil Yachty)'})
confidence =  0.5093457943925234
________________________________
frozenset({'Congratulations', 'iSpy (feat. Lil Yachty)'}) => frozenset({'HUMBLE.'})
confidence =  0.6728395061728395
________________________________
frozenset({'HUMBLE.', 'iSpy (feat. Lil Yachty)'}) => frozenset({'Congratulations'})
confidence =  0.7032258064516129
________________________________
frozenset({'HUMBLE.', 'DNA.'}) => frozenset({'Mask Off'})
confidence =  0.5263157894736842
________________________________
frozenset({'DNA.', 'Mask Off'}) => frozenset({'HUMBLE.'})
confidence =  0.9090909090909092
________________________________
frozenset({'Tunnel Vision', 'HUMBLE.'}) => frozenset({'XO TOUR Llif3'})
confidence =  0.7338129496402879
________________________________
frozenset({'Tunnel Vision', 'XO TOUR Llif3'}) => frozenset({'HUMBLE.'})
confidence =  0.7500000000000001
________________________________
frozenset({'HUMBLE.', 'XO 

---